In [21]:
#We load in the specific data needed
import polars as pl
import numpy as np
import os
from pathlib import Path
import pandas as pd
from datetime import date
import datetime
import hvplot.polars
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import bisplrep, bisplev
from datetime import timedelta
from patsy import dmatrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


In [22]:
data_train_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_scaled_p.parquet')
data_train_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_scaled_c.parquet')

data_val_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_val_tech_scaled_p.parquet')
data_val_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_val_tech_scaled_c.parquet')

data_test_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_scaled_p.parquet')
data_test_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_scaled_c.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

data_train = pd.concat([data_train_p, data_train_c], axis=0)
data_val = pd.concat([data_val_p, data_val_c], axis=0)
data_test = pd.concat([data_test_p, data_test_c], axis=0)

# List of columns to drop
columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:])
# columns_to_drop = ['trading_days_till_exp']

# Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
data_val = data_val.drop(columns=columns_to_drop, errors='ignore')
data_test = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_val = data_val[data_train.columns]
data_test = data_test[data_train.columns]

data_test



,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,5_day_rolling_return_stock,ASK,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
36352,AAPL,2021-01-04,P,0.019841,0.622737,-1.977,0.452074,0.404451,0.092909,99.752109,...,0.001770,0.032360,20.280237,-0.004600,21.627287,19.517935,20.380034,676.851764,2.706552,1.849205
36353,AAPL,2021-01-04,P,0.019841,0.598160,-1.835,0.418943,0.383928,0.092909,99.752109,...,0.001770,0.032360,20.280237,-0.004600,21.627287,19.517935,20.380034,676.851764,2.706552,1.849205
36354,AAPL,2021-01-04,P,0.019841,0.570362,-1.694,0.403192,0.374351,0.092909,99.752109,...,0.001770,0.032360,20.280237,-0.004600,21.627287,19.517935,20.380034,676.851764,2.706552,1.849205
36355,AAPL,2021-01-04,P,0.019841,0.549241,-1.554,0.388048,0.365382,0.092909,99.752109,...,0.001770,0.032360,20.280237,-0.004600,21.627287,19.517935,20.380034,676.851764,2.706552,1.849205
36356,AAPL,2021-01-04,P,0.019841,0.526871,-1.415,0.360382,0.349066,0.092909,99.752109,...,0.001770,0.032360,20.280237,-0.004600,21.627287,19.517935,20.380034,676.851764,2.706552,1.849205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36347,TSLA,2021-12-30,C,0.007937,0.742669,1.556,0.773147,0.718058,0.292655,822.255303,...,0.390523,0.706535,12.255009,-0.005836,13.997953,11.876221,13.695484,635.297620,-0.271641,1.960845
36348,TSLA,2021-12-30,C,0.007937,0.764381,1.658,0.789361,0.735985,0.292655,822.255303,...,0.390523,0.706535,12.255009,-0.005836,13.997953,11.876221,13.695484,635.297620,-0.271641,1.960845
36349,TSLA,2021-12-30,C,0.007937,0.786317,1.760,0.807648,0.754168,0.292655,822.255303,...,0.390523,0.706535,12.255009,-0.005836,13.997953,11.876221,13.695484,635.297620,-0.271641,1.960845
36350,TSLA,2021-12-30,C,0.007937,0.798978,1.861,0.821686,0.773210,0.292655,822.255303,...,0.390523,0.706535,12.255009,-0.005836,13.997953,11.876221,13.695484,635.297620,-0.271641,1.960845


In [23]:
# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare validation data for Call and Put options
data_validate_c = data_val[data_val['cp_flag'] == 'C']
data_validate_p = data_val[data_val['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

In [ ]:
import keras.backend as K
from keras import regularizers
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import RMSprop
from keras.optimizers import Adam

# define the model
def create_model(bias, activity):
  model = Sequential()
  model.add(Dense(units=32,
                  activation='relu',
                  input_dim= train_x.shape[1],
                  # # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5),
                  bias_regularizer= regularizers.L2(bias),
                  activity_regularizer= regularizers.L2(activity)
                  ))
  model.add(Dense(units=32, activation= 'relu',
                  bias_regularizer=regularizers.L2(bias),
                  activity_regularizer=regularizers.L2(activity) #activation
                  ))
  model.add(Dense(units=1, activation= 'linear')) #miss linear

  # compile the model
  model.compile(optimizer= RMSprop(learning_rate=0.1), #'rmsprop',#Optimizer_trial,
                loss= 'mean_squared_error',
                metrics=['mse'])
  return model

In [25]:
import tensorflow as tf
print(tf.__version__)
import time
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from keras_tuner import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from keras_tuner import KerasRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping

def create_model(hidden_layers=1, units=64, activation='relu', learning_rate=0.001):
    """
    Function to create a Keras Sequential model with the given hyperparameters.
    """
    model = Sequential()
    model.add(Dense(units=units, activation=activation, input_dim=X_train.shape[1]))
    
    for _ in range(hidden_layers - 1):
        model.add(Dense(units=units, activation=activation))
        
    model.add(Dense(1))  # Output layer

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

def prepare_data_with_gridsearch_nn(train_data, validate_data, test_data, option_type, verbose=True):
    """
    Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
    retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test) for Neural Networks.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset (Year 1).
    validate_data (pd.DataFrame): The validation dataset (Year 2).
    test_data (pd.DataFrame): The testing dataset (Year 3).
    option_type (str): Call or Put option type for labeling the print output.
    verbose (bool): If True, prints progress information for hyperparameter tuning.
    """
    # Prepare the train, validation, and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
    y_train = train_data['impl_volatility']  # Training target (Year 1)

    X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
    y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
    y_test = test_data['impl_volatility']  # Test target (Year 3)

    # Define the hyperparameter grid for NN
    param_grid = {
        'hidden_layers': [1, 2, 3],                # Number of hidden layers
        'units': [32, 64, 128],                    # Number of units per layer
        'activation': ['relu', 'tanh'],            # Activation function
        'learning_rate': [0.001, 0.01],            # Learning rate for Adam optimizer
        'batch_size': [32, 64],                    # Batch size
        'epochs': [50, 100],                       # Number of epochs
    }

    # Generate all combinations of hyperparameters
    param_combinations = list(itertools.product(
        param_grid['hidden_layers'], 
        param_grid['units'], 
        param_grid['activation'], 
        param_grid['learning_rate'], 
        param_grid['batch_size'], 
        param_grid['epochs']
    ))

    total_combinations = len(param_combinations)
    
    # Initialize variables to store the best model and best score
    best_rmse_val = np.inf
    best_params = None
    best_nn_model = None

    print(f"Running manual hyperparameter tuning for {option_type} Options with Neural Networks...")
    
    # Iterate over all hyperparameter combinations with progress tracking
    for i, (hidden_layers, units, activation, learning_rate, batch_size, epochs) in enumerate(param_combinations):
        start_time = time.time()
        
        # Create the Keras model with the current set of hyperparameters
        model = create_model(hidden_layers=hidden_layers, units=units, activation=activation, learning_rate=learning_rate)

        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model on the training data (Year 1)
        history = model.fit(X_train, y_train, 
                            validation_data=(X_validate, y_validate),
                            batch_size=batch_size, 
                            epochs=epochs, 
                            verbose=0, 
                            callbacks=[early_stopping])

        # Validate the model on the validation data (Year 2)
        y_val_pred = model.predict(X_validate)
        rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

        # Track the best performing hyperparameters based on validation RMSE
        if rmse_val < best_rmse_val:
            best_rmse_val = rmse_val
            best_params = {
                'hidden_layers': hidden_layers,
                'units': units,
                'activation': activation,
                'learning_rate': learning_rate,
                'batch_size': batch_size,
                'epochs': epochs
            }
            best_nn_model = model

        # Verbose output to track progress
        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
            print(f"Current RMSE (Validation): {rmse_val:.4f}")
            print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
    print(f"\nBest Parameters for {option_type} Options with Neural Networks: {best_params}")
    
    # Retrain the model on combined Year 1 (train) and Year 2 (validation)
    print("Retraining the model on Year 1 and Year 2 combined...")
    X_combined = pd.concat([X_train, X_validate])
    y_combined = pd.concat([y_train, y_validate])
    best_nn_model.fit(X_combined, y_combined, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=0)

    # In-sample (combined Year 1 + Year 2) predictions
    y_combined_pred = best_nn_model.predict(X_combined)

    # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
    rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
    r2_combined = r2_score(y_combined, y_combined_pred)
    
    print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
    print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
    print(f"R² (Training + Validation): {r2_combined:.4f}")

    # After retraining, evaluate performance on the test data (Year 3)
    y_test_pred = best_nn_model.predict(X_test)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the final results
    print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch_nn(data_train_c, data_validate_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch_nn(data_train_p, data_validate_p, data_test_p, 'Put')


2.17.0


ImportError: cannot import name 'KerasRegressor' from 'keras_tuner' (/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/myenv/lib/python3.12/site-packages/keras_tuner/__init__.py)